# Post-competition Reflection:

#### This notebook is an ensemble of EfficientNet and ResNext.
#### During the competition I experimented the following:
- The EfficientNet models trained by myself (10-fold CV, fine tuned augmentation, Label Smoothing parameter = 0.2) had a 0.902 Pub score
- The ResNext (no TTA) models were borrowed and had a 0.898 Pub score
- 50% weighting on EfficientNet output and 50% weighting on ResNext (no TTA) output had a 0.902 Pub score (Version 2 of this notebook)
- 70% weighting on EfficientNet output and 30% weighting on ResNext (no TTA) output also had a 0.902 Pub score (Version 4 of this notebook)

#### I ended up choosing, as my two final submissions, two variations of the popular public EfficientNet and ResNext ensemble notebook that had a 0.903 Pub score (0.903 was too attractive). I ignored the following two findings:
- EfficientNet in the popular 0.903 Pub score notebook standalone had a relatively low Pub score of 0.896
- By changing the EfficientNet and ResNext ensemble weights in the popular 0.903 Pub score notebook, the Pub score changed a lot (implied potential overfitting on the public test dataset)

#### Finally I didn't get into the top 10%. If I were to use my own ensemble, Version 2 would lift me to nearly the top of the sliver medal range, and Version 4 would lift me to the bottom of the gold medal range. The lesson for me is to use one submission for the best CV score solution, and use the other submission for the best public score solution.

#### Again we can see that ensemble of very different models can be really helpful, even if ResNext is weaker in this case.

#### I added TTA for the ResNext in the most recent version of this notebook.

### Original Notebooks:
- https://www.kaggle.com/mekhdigakhramanian/pytorch-efficientnet-baseline-inference-tta
- https://www.kaggle.com/piantic/no-tta-cassava-resnext50-32x4d-inference-lb0-903
- https://www.kaggle.com/kanruwang/ensemble-efficientnet-and-resnext-inference

# EfficientNet

In [ ]:
package_path = '../input/pytorch-image-models/pytorch-image-models-master'
import sys; sys.path.append(package_path)

In [ ]:
from datetime import datetime
from glob import glob
from scipy.ndimage.interpolation import zoom
from scipy.special import softmax
from skimage import io
from sklearn import metrics
from sklearn.metrics import log_loss
from sklearn.metrics import roc_auc_score, log_loss
from sklearn.model_selection import GroupKFold, StratifiedKFold
from torch import nn
from torch.cuda.amp import autocast, GradScaler
from torch.utils.data import Dataset, DataLoader
from torch.utils.data.sampler import SequentialSampler, RandomSampler
from torchvision import transforms
from tqdm import tqdm
import copy
import cv2
import joblib
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import pydicom
import random
import sklearn
import time
import timm # from efficientnet_pytorch import EfficientNet
import torch
import torchvision
import warnings

In [ ]:
CFG = {
    'fold_num': 7,
    'seed': 719,
    'model_arch': 'tf_efficientnet_b3_ns',
    'img_size': 512,
    'epochs': 32,
    'train_bs': 32,
    'valid_bs': 32,
    'lr': 1e-4,
    'num_workers': 4,
    'accum_iter': 1, # suppoprt to do batch accumulation for backprop with effectively larger batch size
    'verbose_step': 1,
    'device': 'cuda:0',
    'tta': 8
}

In [ ]:
train = pd.read_csv('../input/cassava-leaf-disease-classification/train.csv')
train.head()

In [ ]:
train.label.value_counts()

In [ ]:
submission = pd.read_csv('../input/cassava-leaf-disease-classification/sample_submission.csv')
submission.head()

### Helper Functions

In [ ]:
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True
    
def get_img(path):
    im_bgr = cv2.imread(path)
    im_rgb = im_bgr[:, :, ::-1]
    return im_rgb

img = get_img('../input/cassava-leaf-disease-classification/train_images/1000015157.jpg')
plt.imshow(img)
plt.show()

### Dataset

In [ ]:
class CassavaDataset(Dataset):
    def __init__(
        self, df, data_root, transforms=None, output_label=True
    ):
        
        super().__init__()
        self.df = df.reset_index(drop=True).copy()
        self.transforms = transforms
        self.data_root = data_root
        self.output_label = output_label
    
    def __len__(self):
        return self.df.shape[0]
    
    def __getitem__(self, index: int):
        
        # get labels
        if self.output_label:
            target = self.df.iloc[index]['label']
          
        path = "{}/{}".format(self.data_root, self.df.iloc[index]['image_id'])
        
        img  = get_img(path)
        
        if self.transforms:
            img = self.transforms(image=img)['image']
            
        # do label smoothing
        if self.output_label == True:
            return img, target
        else:
            return img

### Define Train\Validation Image Augmentations

In [ ]:
from albumentations import (
    HorizontalFlip, VerticalFlip, IAAPerspective, ShiftScaleRotate, CLAHE, RandomRotate90,
    Transpose, ShiftScaleRotate, Blur, OpticalDistortion, GridDistortion,
    HueSaturationValue, IAAAdditiveGaussianNoise, GaussNoise, MotionBlur, MedianBlur,
    IAAPiecewiseAffine, RandomResizedCrop, IAASharpen, IAAEmboss,
    RandomBrightnessContrast, Flip, OneOf, Compose, Normalize, Cutout, CoarseDropout,
    ShiftScaleRotate, CenterCrop, Resize
)

from albumentations.pytorch import ToTensorV2

def get_inference_transforms():
    return Compose([
        RandomResizedCrop(CFG['img_size'], CFG['img_size']),
        Transpose(p=0.5),
        HorizontalFlip(p=0.5),
        VerticalFlip(p=0.5),
        ShiftScaleRotate(
            shift_limit=0.025,
            scale_limit=0.05,
            rotate_limit=20,
            p=0.5
        ),
        HueSaturationValue(
            hue_shift_limit=0.2,
            sat_shift_limit=0.2,
            val_shift_limit=0.2,
            p=0.5
        ),
        RandomBrightnessContrast(
            brightness_limit=(-0.1,0.1),
            contrast_limit=(-0.1, 0.1),
            p=0.5
        ),
        Normalize(
            mean=[0.485, 0.456, 0.406],
            std=[0.229, 0.224, 0.225],
            max_pixel_value=255.0,
            p=1.0
        ),
        ToTensorV2(p=1.0)
    ], p=1.)

### Model

In [ ]:
class CassvaImgClassifier(nn.Module):
    def __init__(self, model_arch, n_class, pretrained=False):
        super().__init__()
        self.model = timm.create_model(model_arch, pretrained=pretrained)
        n_features = self.model.classifier.in_features
        self.model.classifier = nn.Linear(n_features, n_class)
        
    def forward(self, x):
        x = self.model(x)
        return x

### Main Loop

**TTA is done within `inference_one_epoch`**

In [ ]:
def inference_one_epoch(model, data_loader, device):
    model.eval()
    image_preds_all = []
    pbar = tqdm(enumerate(data_loader), total=len(data_loader))
    for step, (imgs) in pbar:
        imgs = imgs.to(device).float()
        image_preds = model(imgs)
        image_preds_all += [torch.softmax(image_preds, 1).detach().cpu().numpy()]
    image_preds_all = np.concatenate(image_preds_all, axis=0)
    return image_preds_all

In [ ]:
model_path = [
    "../input/cassava-10-fold-label-smoothing-02/cassava_model_10_fold_labelsmoothing_0.2_small/tf_efficientnet_b3_ns_fold_0_5",
    "../input/cassava-10-fold-label-smoothing-02/cassava_model_10_fold_labelsmoothing_0.2_small/tf_efficientnet_b3_ns_fold_1_9",
    "../input/cassava-10-fold-label-smoothing-02/cassava_model_10_fold_labelsmoothing_0.2_small/tf_efficientnet_b3_ns_fold_2_9",
    "../input/cassava-10-fold-label-smoothing-02/cassava_model_10_fold_labelsmoothing_0.2_small/tf_efficientnet_b3_ns_fold_3_4",
    "../input/cassava-10-fold-label-smoothing-02/cassava_model_10_fold_labelsmoothing_0.2_small/tf_efficientnet_b3_ns_fold_4_3",
    "../input/cassava-10-fold-label-smoothing-02/cassava_model_10_fold_labelsmoothing_0.2_small/tf_efficientnet_b3_ns_fold_7_9",
    "../input/cassava-10-fold-label-smoothing-02/cassava_model_10_fold_labelsmoothing_0.2_small/tf_efficientnet_b3_ns_fold_8_8",
]

In [ ]:
if __name__ == '__main__':
    seed_everything(CFG['seed'])
    tst_preds_all_folds = []
    for fold in range(CFG['fold_num']):
        test = pd.DataFrame()
        test['image_id'] = sorted(list(
            os.listdir('../input/cassava-leaf-disease-classification/test_images/')
        ))
        test_ds = CassavaDataset(
            test,
            '../input/cassava-leaf-disease-classification/test_images/',
            transforms=get_inference_transforms(),
            output_label=False
        )
        tst_loader = torch.utils.data.DataLoader(
            test_ds, 
            batch_size=CFG['valid_bs'],
            num_workers=CFG['num_workers'],
            shuffle=False,
            pin_memory=False,
        )
        device = torch.device(CFG['device'])
        model = CassvaImgClassifier(CFG['model_arch'], train.label.nunique()).to(device)
        tst_preds = []
        model.load_state_dict(torch.load(model_path[fold]))
        with torch.no_grad():
            for _ in range(CFG['tta']):
                tst_preds += [
                    inference_one_epoch(model, tst_loader, device)
                ]
        # Average over TTA
        tst_preds = np.mean(tst_preds, axis=0)

        # Inference of this model is done; append the results
        tst_preds_all_folds.append(tst_preds)

        del model
        torch.cuda.empty_cache()

### Clean Up Variables

In [ ]:
variable_list = %who_ls
for _ in variable_list:
    if _ is not "tst_preds_all_folds":
        del globals()[_]

%who_ls

<br>
<br>

# ResNext

In [ ]:
# ====================================================
# Library
# ====================================================
import sys
sys.path.append('../input/pytorch-image-models/pytorch-image-models-master')

from albumentations.pytorch import ToTensorV2
from collections import defaultdict, Counter
from contextlib import contextmanager
from functools import partial
from pathlib import Path
from PIL import Image
from scipy.special import softmax
from sklearn import preprocessing
from sklearn.metrics import accuracy_score
from sklearn.model_selection import StratifiedKFold
from torch.nn.parameter import Parameter
from torch.optim import Adam, SGD
from torch.optim.lr_scheduler import CosineAnnealingWarmRestarts, CosineAnnealingLR, ReduceLROnPlateau
from torch.utils.data import DataLoader, Dataset
from tqdm.auto import tqdm
import albumentations as A
import cv2
import math
import numpy as np
import os
import pandas as pd
import random
import scipy as sp
import shutil
import time
import timm
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision.models as models
import warnings

warnings.filterwarnings('ignore')
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

### Directory settings

In [ ]:
OUTPUT_DIR = './'
MODEL_DIR = '../input/cassava-resnext/'
if not os.path.exists(OUTPUT_DIR):
    os.makedirs(OUTPUT_DIR)
    
TEST_PATH = '../input/cassava-leaf-disease-classification/test_images'

### CFG

In [ ]:
# ====================================================
# CFG for Resnext
# ====================================================
class CFG:
    debug=False
    num_workers=8
    model_name='resnext50_32x4d'
    size=512
    batch_size=32 ######## SET THIS TO 2 WHEN DEBUGGING ########
    seed=2020
    target_size=5
    target_col='label'
    n_fold=5
    trn_fold=[0, 1, 2, 3, 4]
    inference=True
    tta=8

### Utils

In [ ]:
# ====================================================
# Utils for Resnext
# ====================================================
def get_score(y_true, y_pred):
    return accuracy_score(y_true, y_pred)


@contextmanager
def timer(name):
    t0 = time.time()
    LOGGER.info(f'[{name}] start')
    yield
    LOGGER.info(f'[{name}] done in {time.time() - t0:.0f} s.')


def init_logger(log_file=OUTPUT_DIR+'inference.log'):
    from logging import getLogger, INFO, FileHandler,  Formatter,  StreamHandler
    logger = getLogger(__name__)
    logger.setLevel(INFO)
    handler1 = StreamHandler()
    handler1.setFormatter(Formatter("%(message)s"))
    handler2 = FileHandler(filename=log_file)
    handler2.setFormatter(Formatter("%(message)s"))
    logger.addHandler(handler1)
    logger.addHandler(handler2)
    return logger

def seed_torch(seed=42):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True

seed_torch(seed=CFG.seed)

### Data Loading

In [ ]:
test = pd.read_csv('../input/cassava-leaf-disease-classification/sample_submission.csv')
test['filepath'] = test.image_id.apply(lambda x: os.path.join('../input/cassava-leaf-disease-classification/test_images', f'{x}'))

############## FOR DEBUGGING ################
# test = pd.concat([test, test, test, test, test, test])
#############################################

### Dataset

In [ ]:
# ====================================================
# Dataset for Resnext
# ====================================================
class TestDataset(Dataset):
    def __init__(self, df, transform=None):
        self.df = df
        self.file_names = df['image_id'].values
        self.transform = transform
        
    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        file_name = self.file_names[idx]
        file_path = f'{TEST_PATH}/{file_name}'
        image = cv2.imread(file_path)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        if self.transform:
            augmented = self.transform(image=image)
            image = augmented['image']
        return image

In [ ]:
# ====================================================
# Transforms for Resnext
# ====================================================
def get_transforms(*, data):
    if data == 'valid':
        return A.Compose([
            A.Resize(CFG.size, CFG.size),
            A.Transpose(p=0.5),
            A.HorizontalFlip(p=0.5),
            A.VerticalFlip(p=0.5),
            A.Normalize(
                mean=[0.485, 0.456, 0.406],
                std=[0.229, 0.224, 0.225],
            ),
            ToTensorV2()
        ])

### MODELS

In [ ]:
# ====================================================
# ResNext Model
# ====================================================
class CustomResNext(nn.Module):
    def __init__(self, model_name='resnext50_32x4d', pretrained=False):
        super().__init__()
        self.model = timm.create_model(model_name, pretrained=pretrained)
        n_features = self.model.fc.in_features
        self.model.fc = nn.Linear(n_features, CFG.target_size)

    def forward(self, x):
        x = self.model(x)
        return x


### Helper functions

In [ ]:
# ====================================================
# Helper functions for Resnext
# ====================================================
def load_state(model_path):
    model = CustomResNext(CFG.model_name, pretrained=False)
    try:  # single GPU model_file
        model.load_state_dict(torch.load(model_path)['model'], strict=True)
        state_dict = torch.load(model_path)['model']
    except:  # multi GPU model_file
        state_dict = torch.load(model_path)['model']
        state_dict = {k[7:] if k.startswith('module.') else k: state_dict[k] for k in state_dict.keys()}

    return state_dict

def inference(model, states, test_loader, device):
    tta_probs = []
    for e in range(CFG.tta):
        model.to(device)
        tk0 = tqdm(enumerate(test_loader), total=len(test_loader))
        probs = []
        for i, (images) in tk0:
            images = images.to(device)
            avg_preds = []
            for state in states:
                model.load_state_dict(state)
                model.eval()
                with torch.no_grad():
                    y_preds = model(images)
                avg_preds.append(y_preds.softmax(1).to('cpu').numpy())
            avg_preds = np.mean(avg_preds, axis=0)
            probs.append(avg_preds)
        probs = np.concatenate(probs)
        tta_probs.append(probs)
    tta_probs = np.mean(tta_probs, axis=0)
    return tta_probs

### Inference

In [ ]:
# ====================================================
# inference
# ====================================================

model = CustomResNext(CFG.model_name, pretrained=False)
states = [load_state(MODEL_DIR+f'{CFG.model_name}_fold{fold}.pth') for fold in CFG.trn_fold]
test_dataset = TestDataset(test, transform=get_transforms(data='valid'))
test_loader = DataLoader(test_dataset, batch_size=CFG.batch_size, shuffle=False, 
                         num_workers=CFG.num_workers, pin_memory=True)
predictions = inference(model, states, test_loader, device)

<br>
<br>

# Final Combine

In [ ]:
submission = test[["image_id"]]


submission["label"] = (
    np.mean(tst_preds_all_folds, axis=0) * 0.7
    + predictions * 0.3
).argmax(1)

In [ ]:
submission.head()

In [ ]:
submission.to_csv("submission.csv", index=False)